Importo librerías

In [1]:
import pandas as pd #manejo de datos
import os #Para trabajar con los directorios del equipo
import locale #Para cambiar el idioma local a español


In [2]:
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8') #Set en español

'es_ES.UTF-8'

In [3]:
#Esta parte no es necesaria pero es para que no aparezcan los warning 
import warnings 

# Configurar los warnings para que se ignoren
warnings.filterwarnings("ignore")

Subo archivo

In [4]:
data = pd.read_excel('C:/OTMX/Documentation/DATAMARSH.xlsx')

In [5]:
data.columns #veo las columnas

Index(['MES DESDE', 'MES FACTURA', 'NOMBRE AGENCIA', 'NOMBRE CORTO', 'ESTADO',
       'GRUPO', 'SEGMENTO GRUPO', 'RAMO', 'SUB RAMO', 'DESCRIPCION FORMA PAGO',
       'FRECUENCIA PAGO', 'UNIDAD', 'NOMBRE DEL PRODUCTO', 'NOMBRE DEL PLAN',
       'NOMBRE ZONA', 'TIPO MOV.POLIZA', 'TIPO MOVIMIENTO', 'NUMERO_DE_POLIZA',
       'SEGMENTO POLIZA', 'FECHA_FACTURA', 'DESDE', 'HASTA', 'FECHA EMISION',
       'POR_COMISION', 'POR_COMISION_VIDA', 'SECUENCIA', 'NOMBRE NEGOCIO',
       'PRIMA', 'MONTO', 'TOTAL_COMISION', 'COMISION_VIDA', 'COMISION',
       'SUMA ASEGURADA', 'PRODUCTO', 'SUBSEGEMENTO'],
      dtype='object')

In [6]:
subconjunto = data[['DESDE','NOMBRE CORTO','TIPO MOVIMIENTO','NUMERO_DE_POLIZA','MONTO']] #hago un subconjunto

In [7]:
subconjunto['DESDE'] = subconjunto['DESDE'].astype(str) #vuelvo a str la columna de fecha solamente para mostrar más adelante cómo volverla a fecha 
#no necesario

In [8]:
subconjunto.dtypes #miro los tipos 

DESDE                object
NOMBRE CORTO         object
TIPO MOVIMIENTO      object
NUMERO_DE_POLIZA     object
MONTO               float64
dtype: object

In [9]:
subconjunto['DESDE'] = pd.to_datetime(subconjunto['DESDE']) #vuelvo la columna DESDE a tipo fecha

In [10]:
subconjunto.dtypes #miro los tipos

DESDE               datetime64[ns]
NOMBRE CORTO                object
TIPO MOVIMIENTO             object
NUMERO_DE_POLIZA            object
MONTO                      float64
dtype: object

In [11]:
#crear una nueva columna año
subconjunto['AÑO'] = subconjunto['DESDE'].dt.strftime('%Y')

In [12]:
#crear una nueva columna de mes
subconjunto['MES'] = subconjunto['DESDE'].dt.strftime('%m')

In [13]:
import calendar #con esta librería podemos convertir los números del 1 al 12 a los meses en nombres, ejemplo: el 2 lo convierte en febrero
#esta librería convierte los nombres al mes en el idioma en el que esté tu trabajo local, por ese motivo se coloca en español

def convertir_mes(numero_mes): #se crea una función a la que le paso el número del mes y por medio del método month_name de calendar me vuelve el número al nombre
    return calendar.month_name[numero_mes]

In [14]:
#como quiero usar esta columna para convertir los números a mes, debo pasarla primero a tipo int porque calendar no funciona si es str
subconjunto['MES'] = subconjunto['MES'].astype(int)

In [16]:
subconjunto['NOMBRE_MES'] = subconjunto['MES'].apply(convertir_mes) #creo la columna "NOMBRE_MES" y le aplico la función a la columna "MES"

In [17]:
subconjunto['DESDE'] = subconjunto['DESDE'].dt.strftime('%Y-%m-%d') #Puedo convertir la columna DESDE al formato de solamente "AAAA-mm-dd" pero en str porque aveces en el Excel no queremos formatos tan largos que incluyan tiempo

In [18]:
#FILTROS
df_filtrado = subconjunto[subconjunto['TIPO MOVIMIENTO']=="REN"] #Aplico filtro de Renovación en TIPO MOVIMIENTO

In [19]:
df_filtrado2 = subconjunto[(subconjunto['TIPO MOVIMIENTO']=="REN")&(subconjunto['NOMBRE CORTO']=="ASSA")] #aplico dos filtros, REN y ASSA

In [20]:
subconjunto.isna().sum() #Con esto me doy cuenta cuántos NA hay por columna

DESDE               0
NOMBRE CORTO        1
TIPO MOVIMIENTO     0
NUMERO_DE_POLIZA    0
MONTO               0
AÑO                 0
MES                 0
NOMBRE_MES          0
dtype: int64

In [21]:
subconjunto = subconjunto.dropna(subset=['NOMBRE CORTO']) #borro las filas que tengan NA en la columna NOMBRE CORTO

In [22]:
df_filtrado3 = subconjunto[subconjunto['NOMBRE CORTO'].str.contains('SURA')] #Hago un filtro por la columna NOMBRE CORTO de lo que contenga 'SURA' es decir si el valor es
#"SURAMERICANA" igual lo va a filtrar porque contiene SURA, esta línea funciona si no hay NA en la misma

EXPORTAR ARCHIVOS

In [23]:
#UNICA HOJA
subconjunto.to_excel('nombrearchivo.xlsx', sheet_name='data', index=False) #subconjunto es el dataframe, nombrearchivo el nombre que quiero ponerle
#'data' es el nombre de la hoja del excel donde estará la info, index=False para que no exporte los resultados con el index de Python

In [24]:
#VARIAS HOJAS
#si quiero en la primer hoja exportar data y en la segunda subconjunto
import xlsxwriter

os.chdir('C:/OTMX/Documentation') #directorio donde quiero que exporte los resultados en caso de ser diferente
#al directorio en el que se encuentra el 

# cree un escritor 
writer = pd.ExcelWriter('variashojas.xlsx')

#escribe cada DataFrame en una hoja específica
data.to_excel (writer, sheet_name='data', index=False)
subconjunto.to_excel (writer, sheet_name='subconjunto',index=False)

#cierre el escritor de Pandas Excel y genere el archivo de Excel
writer.close()